In [4]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

# config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf", 
    # quantization_config=config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_fast=True)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

c:\Users\rohit\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rohit\.cache\huggingface\hub\models--meta-llama--Llama-2-7b-chat-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

Some parameters are on the meta device because they were offloaded to the cpu and disk.


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset 

dataset = load_dataset("glue", "sst2", split="validation")

In [6]:
print(f"Device: {model.device}")
print(f"Validation samples: {len(dataset)}")

Device: cuda:0
Validation samples: 872


In [9]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

In [22]:
accuracy = 0
for i, data in enumerate(dataset.select(range(200))):
    sentence = data['sentence']
    prompt = f"""Classify the text into negative or positive. Text: {sentence} Sentiment: """.strip()
    tokens = tokenizer(prompt, return_tensors="pt", padding=True)
    tokens = tokens.input_ids.to(model.device)
    out = model.generate(tokens, pad_token_id=tokenizer.pad_token_id, max_new_tokens=3)
    print(sentence)
    predicted = tokenizer.decode(out[0, -3:])
    predicted = predicted.strip().lower()
    if data['label'] == 0 and "negative" in predicted:
        accuracy += 1
    elif data['label'] == 1 and "positive" in predicted:
        accuracy += 1
    print(f"Predicted: {tokenizer.decode(out[0, -3:])} Actual: {data['label']} Accuracy: {accuracy} / {i + 1}")
     

it 's a charming and often affecting journey . 
Predicted: Positive. Actual: 1 Accuracy: 1 / 1
unflinchingly bleak and desperate 
Predicted: : negative</s> Actual: 0 Accuracy: 2 / 2
allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . 
Predicted: positive.</s> Actual: 1 Accuracy: 3 / 3
the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . 
Predicted: Positive
 Actual: 1 Accuracy: 4 / 4
it 's slow -- very , very slow . 
Predicted: negative.</s> Actual: 0 Accuracy: 5 / 5
although laced with humor and a few fanciful touches , the film is a refreshingly serious look at young women . 
Predicted: positive.</s> Actual: 1 Accuracy: 6 / 6
a sometimes tedious film . 
Predicted: neutral.</s> Actual: 0 Accuracy: 6 / 7
or doing last year 's taxes with your ex-wife . 
Predicted: Negative. Actual: 0 Accuracy: 7 / 8
you do n't have to know about music to appreciate the film 's easygoing

KeyboardInterrupt: 